In [1]:
!export RAPIDS_NO_INITIALIZE="1"
!export CUDF_SPILL="1"
!export LIBCUDF_CUFILE_POLICY="OFF"

from cugraph_bulk_sampling import start_dask_client, benchmark_cugraph_bulk_sampling, load_disk_dataset, construct_graph
from cugraph_bulk_sampling import sample_graph
import os

# Setup Cluster

In [2]:
dask_worker_devices='0,1,2,3,4,5,6,7'

In [3]:
client, cluster = start_dask_client(dask_worker_devices=dask_worker_devices,
                                    jit_unspill=False,
                                    rmm_pool_size=28e9,
                                    rmm_async=True)


Dask client/cluster created using LocalCUDACluster


# Setup Benchmark

In [4]:
dataset='ogbn_papers100M'
dataset_root="/datasets/abarghi"
output_root="/tmp/ramdisk"
reverse_edges=True
add_edge_types=False
batch_size=512
seeds_per_call=524288
fanout=[25,25]
replication_factor=2
seed=123

dataset_dir=dataset_root
output_path=output_root
persist=False


In [5]:
dask_edgelist_df, dask_label_df, node_offsets, edge_offsets, total_num_nodes = \
    load_disk_dataset(
        dataset,
        dataset_dir=dataset_dir,
        reverse_edges=reverse_edges,
        replication_factor=replication_factor,
        persist=False,
        add_edge_types=add_edge_types
    )
num_input_edges = len(dask_edgelist_df)
print(
f"Number of input edges = {num_input_edges:,}"
)

G = construct_graph(
dask_edgelist_df
)
del dask_edgelist_df
print('constructed graph')

Loading edge index for edge type paper__cites__paper
Loading node labels for node type paper (offset=0)
Number of input edges = 3,231,371,744
constructed graph


In [6]:
input_memory = G.edgelist.edgelist_df.memory_usage().sum().compute()
print(f'input memory: {input_memory}')

output_subdir = os.path.join(output_path, f'{dataset}[{replication_factor}]_b{batch_size}_f{fanout}')
os.makedirs(output_subdir, exist_ok=True)

output_sample_path = os.path.join(output_subdir, 'samples')
os.makedirs(output_sample_path,  exist_ok=True)

batches_per_partition = 200_000 // batch_size

/opt/conda/envs/rapids/lib/python3.10/site-packages/cudf/core/index.py:3139: FutureWarning: cudf.StringIndex is deprecated and will be removed from cudf in a future version. Use cudf.Index with the appropriate dtype instead.
  warnings.warn(
/opt/conda/envs/rapids/lib/python3.10/site-packages/cudf/core/index.py:3139: FutureWarning: cudf.StringIndex is deprecated and will be removed from cudf in a future version. Use cudf.Index with the appropriate dtype instead.
  warnings.warn(


input memory: 51701947904


# Benchmarking Sample Graph

In [8]:
%%timeit -n30 -r1


execution_time, allocation_counts = sample_graph(
    G,
    dask_label_df,
    output_sample_path,
    seed=seed,
    batch_size=batch_size,
    seeds_per_call=seeds_per_call,
    batches_per_partition=batches_per_partition,
    fanout=fanout,
    persist=persist,
)


created batches
flushed all batches
function:  sample_graph
function args: (<cugraph.structure.graph_classes.MultiGraph object at 0x7f9beb8b1a20>, <dask_cudf.DataFrame | 16 tasks | 8 npartitions>, '/tmp/ramdisk/ogbn_papers100M[2]_b512_f[25, 25]/samples') kwargs: {'seed': 123, 'batch_size': 512, 'seeds_per_call': 524288, 'batches_per_partition': 390, 'fanout': [25, 25], 'persist': False}
execution_time: 2.5521183013916016
allocation_counts:
{   'tcp://127.0.0.1:35729': {   'current_bytes': '15.1MB',
                                 'peak_bytes': '2.9GB',
                                 'total_bytes': '39.6GB'},
    'tcp://127.0.0.1:38199': {   'current_bytes': '269.4MB',
                                 'peak_bytes': '2.8GB',
                                 'total_bytes': '40.0GB'},
    'tcp://127.0.0.1:38959': {   'current_bytes': '116.9MB',
                                 'peak_bytes': '2.9GB',
                                 'total_bytes': '40.1GB'},
    'tcp://127.0.0.1:40829': 